<a href="https://colab.research.google.com/github/sathi26/ML_Project/blob/main/Food_Delivery_Time_Prediction_using_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px

data = pd.read_csv("deliverytime.txt")
print(data.head())

     ID Delivery_person_ID  Delivery_person_Age  Delivery_person_Ratings  \
0  4607     INDORES13DEL02                 37.0                      4.9   
1  B379     BANGRES18DEL02                 34.0                      4.5   
2  5D6D     BANGRES19DEL01                 23.0                      4.4   
3  7A6A    COIMBRES13DEL02                 38.0                      4.7   
4  70A2     CHENRES12DEL01                 32.0                      4.6   

   Restaurant_latitude  Restaurant_longitude  Delivery_location_latitude  \
0            22.745049             75.892471                   22.765049   
1            12.913041             77.683237                   13.043041   
2            12.914264             77.678400                   12.924264   
3            11.003669             76.976494                   11.053669   
4            12.972793             80.249982                   13.012793   

   Delivery_location_longitude Type_of_order Type_of_vehicle  Time_taken(min)  
0     

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12189 entries, 0 to 12188
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           12189 non-null  object 
 1   Delivery_person_ID           12189 non-null  object 
 2   Delivery_person_Age          12188 non-null  float64
 3   Delivery_person_Ratings      12188 non-null  float64
 4   Restaurant_latitude          12188 non-null  float64
 5   Restaurant_longitude         12188 non-null  float64
 6   Delivery_location_latitude   12188 non-null  float64
 7   Delivery_location_longitude  12188 non-null  float64
 8   Type_of_order                12188 non-null  object 
 9   Type_of_vehicle              12188 non-null  object 
 10  Time_taken(min)              12188 non-null  float64
dtypes: float64(7), object(4)
memory usage: 1.0+ MB


In [6]:
data.isnull().sum()

,0
ID,0
Delivery_person_ID,0
Delivery_person_Age,1
Delivery_person_Ratings,1
Restaurant_latitude,1
Restaurant_longitude,1
Delivery_location_latitude,1
Delivery_location_longitude,1
Type_of_order,1
Type_of_vehicle,1


In [7]:
# Set the earth's radius (in kilometers)
R = 6371

# Convert degrees to radians
def deg_to_rad(degrees):
    return degrees * (np.pi/180)

# Function to calculate the distance between two points using the haversine formula
def distcalculate(lat1, lon1, lat2, lon2):
    d_lat = deg_to_rad(lat2-lat1)
    d_lon = deg_to_rad(lon2-lon1)
    a = np.sin(d_lat/2)**2 + np.cos(deg_to_rad(lat1)) * np.cos(deg_to_rad(lat2)) * np.sin(d_lon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

# Calculate the distance between each pair of points
data['distance'] = np.nan

for i in range(len(data)):
    data.loc[i, 'distance'] = distcalculate(data.loc[i, 'Restaurant_latitude'],
                                        data.loc[i, 'Restaurant_longitude'],
                                        data.loc[i, 'Delivery_location_latitude'],
                                        data.loc[i, 'Delivery_location_longitude'])

In [8]:
print(data.head())

     ID Delivery_person_ID  Delivery_person_Age  Delivery_person_Ratings  \
0  4607     INDORES13DEL02                 37.0                      4.9   
1  B379     BANGRES18DEL02                 34.0                      4.5   
2  5D6D     BANGRES19DEL01                 23.0                      4.4   
3  7A6A    COIMBRES13DEL02                 38.0                      4.7   
4  70A2     CHENRES12DEL01                 32.0                      4.6   

   Restaurant_latitude  Restaurant_longitude  Delivery_location_latitude  \
0            22.745049             75.892471                   22.765049   
1            12.913041             77.683237                   13.043041   
2            12.914264             77.678400                   12.924264   
3            11.003669             76.976494                   11.053669   
4            12.972793             80.249982                   13.012793   

   Delivery_location_longitude Type_of_order Type_of_vehicle  Time_taken(min)  \
0    

In [9]:
# Drop rows with missing values in 'Time_taken(min)'
data_cleaned = data.dropna(subset=['Time_taken(min)'])

figure = px.scatter(data_frame = data_cleaned,
                    x="distance",
                    y="Time_taken(min)",
                    size="Time_taken(min)",
                    trendline="ols",
                    title = "Relationship Between Distance and Time Taken")
figure.show()

In [10]:
figure = px.scatter(data_frame = data_cleaned,
                    x="Delivery_person_Age",
                    y="Time_taken(min)",
                    size="Time_taken(min)",
                    color = "distance",
                    trendline="ols",
                    title = "Relationship Between Time Taken and Age")
figure.show()

In [11]:
figure = px.scatter(data_frame = data_cleaned,
                    x="Delivery_person_Ratings",
                    y="Time_taken(min)",
                    size="Time_taken(min)",
                    color = "distance",
                    trendline="ols",
                    title = "Relationship Between Time Taken and Ratings")
figure.show()

In [12]:
fig = px.box(data,
             x="Type_of_vehicle",
             y="Time_taken(min)",
             color="Type_of_order")
fig.show()

In [15]:
# Drop rows with missing values in the columns used for training
data_cleaned_for_training = data_cleaned.dropna(subset=["Delivery_person_Age",
                                                         "Delivery_person_Ratings",
                                                         "distance",
                                                         "Time_taken(min)"])

#splitting data
from sklearn.model_selection import train_test_split
x = np.array(data_cleaned_for_training[["Delivery_person_Age",
                   "Delivery_person_Ratings",
                   "distance"]])
y = np.array(data_cleaned_for_training[["Time_taken(min)"]])

# Reshape input data for LSTM (samples, timesteps, features)
x = x.reshape(x.shape[0], 1, x.shape[1])

xtrain, xtest, ytrain, ytest = train_test_split(x, y,
                                                test_size=0.10,
                                                random_state=42)

# creating the LSTM neural network model
from keras.models import Sequential
from keras.layers import Dense, LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (xtrain.shape[1], xtrain.shape[2])))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 1, 128)         │        67,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 25)             │         1,625 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,643 (463.45 KB)

 Trainable params: 118,643 (463.45 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# training the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(xtrain, ytrain, batch_size=1, epochs=9)

Epoch 1/9
10969/10969 ━━━━━━━━━━━━━━━━━━━━ 59s 5ms/step - loss: 86.5284
Epoch 2/9
10969/10969 ━━━━━━━━━━━━━━━━━━━━ 75s 4ms/step - loss: 69.5259
Epoch 3/9
10969/10969 ━━━━━━━━━━━━━━━━━━━━ 83s 4ms/step - loss: 67.9031
Epoch 4/9
10969/10969 ━━━━━━━━━━━━━━━━━━━━ 80s 4ms/step - loss: 67.7332
Epoch 5/9
10969/10969 ━━━━━━━━━━━━━━━━━━━━ 48s 4ms/step - loss: 65.3394
Epoch 6/9
10969/10969 ━━━━━━━━━━━━━━━━━━━━ 81s 4ms/step - loss: 67.2162
Epoch 7/9
10969/10969 ━━━━━━━━━━━━━━━━━━━━ 80s 4ms/step - loss: 65.6652
Epoch 8/9
10969/10969 ━━━━━━━━━━━━━━━━━━━━ 82s 4ms/step - loss: 65.6405
Epoch 9/9
10969/10969 ━━━━━━━━━━━━━━━━━━━━ 82s 4ms/step - loss: 62.3678


In [18]:
print("Food Delivery Time Prediction")
a = int(input("Age of Delivery Partner: "))
b = float(input("Ratings of Previous Del6iveries: "))
c = int(input("Total Distance: "))

features = np.array([[a, b, c]])
features = features.reshape(features.shape[0], 1, features.shape[1])
print("Predicted Delivery Time in Minutes = ", model.predict(features))

Food Delivery Time Prediction
Age of Delivery Partner: 25
Ratings of Previous Del6iveries: 2.5
Total Distance: 6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
Predicted Delivery Time in Minutes =  [[34.362774]]
